In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, "/cellarold/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages")
import lifelines

from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
from scipy.stats import pearsonr, spearmanr, mannwhitneyu

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [2]:
### STATS ###
import statsmodels.stats.multitest as multi
from matplotlib.collections import PatchCollection

In [3]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [6]:
# tutorials:
    # pandas
    # lifelines

## Get survival information

In [4]:
def make_df(surv,surv_type,rna):
    #get survival dataframe
    surv=pd.read_csv(surv,index_col=0)
    surv=surv.rename(columns={"bcr_patient_barcode":"FID"})
    #get genotypes
    pheno=pd.read_csv(rna,delimiter="\t")
    #combine dataframes
    full_surv=pd.merge(surv[["FID",surv_type,surv_type+".time"]],pheno,on="FID")
    print(full_surv.shape)
    #full_surv=full_surv[full_surv[surv_type+".time"]<1825]
    return(full_surv)

In [5]:
#full rna ~50K
rna=pd.read_csv("C:/Users/symoo/Dropbox/So Youn Moon - Engler Lab/KM Plot/TCGA/all.tumor.rna.tsv",delimiter="\t")
print(rna)

        Unnamed: 0        A1BG      A1CF     A2BP1     A2LD1       A2M  \
0     TCGA-02-0047    125.0070     0.000  244.6300   55.0724  34012.40   
1     TCGA-02-0055    391.8040     0.000  137.3510   84.0140  42876.30   
2     TCGA-02-2483    271.8520     0.000  111.0290   34.5372  21058.50   
3     TCGA-02-2485     83.9429     0.000  257.1430  126.2860   7798.53   
4     TCGA-02-2486    108.2560     0.000    4.2683  190.5300  40971.40   
...            ...         ...       ...       ...       ...       ...   
9697  TCGA-ZS-A9CF  14368.4000  1576.150    0.0000  321.7230  29738.70   
9698  TCGA-ZS-A9CG  18745.0000  2378.060    0.0000  261.8000  10621.90   
9699  TCGA-ZT-A8OM   2840.3900     0.000    0.0000   90.1084  11326.30   
9700  TCGA-ZU-A8S4    169.7020   101.708    0.0000   79.7557   9801.29   
9701  TCGA-ZX-AA5X     79.3117     0.000    0.7734   59.6984   9581.55   

          A2ML1    A4GALT   A4GNT     AAA1  ...    ZWILCH      ZWINT     ZXDA  \
0       41.1814   36.2642  1.2

In [6]:
# selection of HNSC
hnsc=pd.read_csv("C:/Users/symoo/Dropbox/So Youn Moon - Engler Lab/KM Plot/TCGA/TCGA_Case ID_HNSC.csv")
hnsc_pts=hnsc["hnsc_patient_barcode"].tolist()

In [52]:
#loading in the genes
genes=pd.read_csv("genes_oscc_no HSPG2 FSCN1.csv",header=None)
pheno=rna[["Unnamed: 0"]+[x for x in genes[0].tolist() if x in rna.columns.tolist()]]
pheno=pheno.rename(columns={"Unnamed: 0":"FID"})
pheno.to_csv("rna_oscc_gene_no HSPG2 FSCN1.tsv",sep="\t",index=None)

In [53]:
os_surv=make_df("C:/Users/symoo/Dropbox/So Youn Moon - Engler Lab/KM Plot/TCGA/Liu2018.TCGA_survival.csv","OS","rna_oscc_gene_no HSPG2 FSCN1.tsv")

(9676, 8)


In [54]:
pfi_surv=make_df("C:/Users/symoo/Dropbox/So Youn Moon - Engler Lab/KM Plot/TCGA/Liu2018.TCGA_survival.csv","PFI","rna_oscc_gene_no HSPG2 FSCN1.tsv")

(9676, 8)


In [55]:
#subsetting survival data to HNSC
os_surv_hnsc=os_surv[os_surv["FID"].isin(hnsc_pts)]
pfi_surv_hnsc=pfi_surv[pfi_surv["FID"].isin(hnsc_pts)]

In [56]:
#Check if there is null value in the dataframe
print(os_surv_hnsc)
is_NaN=os_surv_hnsc.isnull()
row_has_NaN=is_NaN.any(axis=1)
rows_with_NaN=os_surv_hnsc[row_has_NaN]
print(rows_with_NaN)


               FID   OS  OS.time      SDC4     ICAM1        FN1      RHOD  \
2756  TCGA-4P-AA8J  0.0    102.0   7723.31   3734.40  122122.00  2278.850   
2757  TCGA-BA-4074  1.0    462.0  17101.30   1989.34   95369.30  6075.160   
2758  TCGA-BA-4075  1.0    283.0  29713.00   1375.32   42006.70  4672.650   
2760  TCGA-BA-4077  1.0   1134.0  13060.20   3238.21   15856.10  1421.560   
2774  TCGA-BA-6871  1.0    108.0   1960.77  11332.20   61040.20   402.572   
...            ...  ...      ...       ...       ...        ...       ...   
3243  TCGA-QK-AA3K  0.0    253.0   2242.77   3855.33  210930.00   858.857   
3247  TCGA-T2-A6WZ  1.0    484.0   5483.76   1669.65    6987.68  4541.990   
3270  TCGA-UF-A7JS  1.0    680.0  16310.90   2346.79    3641.26  7357.250   
3273  TCGA-UP-A6WW  0.0    518.0  20406.10   5338.79   22111.50  1330.080   
3275  TCGA-WA-A7H4  0.0    443.0   7153.04   1789.26   52455.90  2401.440   

         ITGAV  
2756   4382.68  
2757  17909.80  
2758   7588.91  
2760   

In [57]:
print(pfi_surv_hnsc)
is_NaN_pfi=pfi_surv_hnsc.isnull()
row_has_NaN_pfi=is_NaN_pfi.any(axis=1)
rows_with_NaN_pfi=pfi_surv_hnsc[row_has_NaN_pfi]
print(rows_with_NaN_pfi)

               FID  PFI  PFI.time      SDC4     ICAM1        FN1      RHOD  \
2756  TCGA-4P-AA8J  0.0     102.0   7723.31   3734.40  122122.00  2278.850   
2757  TCGA-BA-4074  1.0     396.0  17101.30   1989.34   95369.30  6075.160   
2758  TCGA-BA-4075  1.0     236.0  29713.00   1375.32   42006.70  4672.650   
2760  TCGA-BA-4077  1.0    1134.0  13060.20   3238.21   15856.10  1421.560   
2774  TCGA-BA-6871  1.0     108.0   1960.77  11332.20   61040.20   402.572   
...            ...  ...       ...       ...       ...        ...       ...   
3243  TCGA-QK-AA3K  0.0     253.0   2242.77   3855.33  210930.00   858.857   
3247  TCGA-T2-A6WZ  1.0     229.0   5483.76   1669.65    6987.68  4541.990   
3270  TCGA-UF-A7JS  1.0     519.0  16310.90   2346.79    3641.26  7357.250   
3273  TCGA-UP-A6WW  0.0     518.0  20406.10   5338.79   22111.50  1330.080   
3275  TCGA-WA-A7H4  0.0     443.0   7153.04   1789.26   52455.90  2401.440   

         ITGAV  
2756   4382.68  
2757  17909.80  
2758   7588.

In [58]:
os_surv_hnsc=os_surv_hnsc.set_index("FID")
os_surv_hnsc.head()

,OS,OS.time,SDC4,ICAM1,FN1,RHOD,ITGAV
FID,,,,,,,
TCGA-4P-AA8J,0.0,102.0,7723.31,3734.40,122122.0,2278.850,4382.68
TCGA-BA-4074,1.0,462.0,17101.30,1989.34,95369.3,6075.160,17909.80
TCGA-BA-4075,1.0,283.0,29713.00,1375.32,42006.7,4672.650,7588.91
TCGA-BA-4077,1.0,1134.0,13060.20,3238.21,15856.1,1421.560,6872.47
TCGA-BA-6871,1.0,108.0,1960.77,11332.20,61040.2,402.572,2968.81


In [59]:
os_surv_hnsc=os_surv_hnsc.drop("TCGA-CQ-A4CA")

In [60]:
os_surv_hnsc=os_surv_hnsc.reset_index()
print(os_surv_hnsc)

              FID   OS  OS.time      SDC4     ICAM1        FN1      RHOD  \
0    TCGA-4P-AA8J  0.0    102.0   7723.31   3734.40  122122.00  2278.850   
1    TCGA-BA-4074  1.0    462.0  17101.30   1989.34   95369.30  6075.160   
2    TCGA-BA-4075  1.0    283.0  29713.00   1375.32   42006.70  4672.650   
3    TCGA-BA-4077  1.0   1134.0  13060.20   3238.21   15856.10  1421.560   
4    TCGA-BA-6871  1.0    108.0   1960.77  11332.20   61040.20   402.572   
..            ...  ...      ...       ...       ...        ...       ...   
147  TCGA-QK-AA3K  0.0    253.0   2242.77   3855.33  210930.00   858.857   
148  TCGA-T2-A6WZ  1.0    484.0   5483.76   1669.65    6987.68  4541.990   
149  TCGA-UF-A7JS  1.0    680.0  16310.90   2346.79    3641.26  7357.250   
150  TCGA-UP-A6WW  0.0    518.0  20406.10   5338.79   22111.50  1330.080   
151  TCGA-WA-A7H4  0.0    443.0   7153.04   1789.26   52455.90  2401.440   

        ITGAV  
0     4382.68  
1    17909.80  
2     7588.91  
3     6872.47  
4     2

In [61]:
pfi_surv_hnsc=pfi_surv_hnsc.set_index("FID")
pfi_surv_hnsc.head()

,PFI,PFI.time,SDC4,ICAM1,FN1,RHOD,ITGAV
FID,,,,,,,
TCGA-4P-AA8J,0.0,102.0,7723.31,3734.40,122122.0,2278.850,4382.68
TCGA-BA-4074,1.0,396.0,17101.30,1989.34,95369.3,6075.160,17909.80
TCGA-BA-4075,1.0,236.0,29713.00,1375.32,42006.7,4672.650,7588.91
TCGA-BA-4077,1.0,1134.0,13060.20,3238.21,15856.1,1421.560,6872.47
TCGA-BA-6871,1.0,108.0,1960.77,11332.20,61040.2,402.572,2968.81


In [62]:
pfi_surv_hnsc=pfi_surv_hnsc.drop("TCGA-CQ-A4CA")

In [63]:
pfi_surv_hnsc=pfi_surv_hnsc.reset_index()
print(pfi_surv_hnsc)

              FID  PFI  PFI.time      SDC4     ICAM1        FN1      RHOD  \
0    TCGA-4P-AA8J  0.0     102.0   7723.31   3734.40  122122.00  2278.850   
1    TCGA-BA-4074  1.0     396.0  17101.30   1989.34   95369.30  6075.160   
2    TCGA-BA-4075  1.0     236.0  29713.00   1375.32   42006.70  4672.650   
3    TCGA-BA-4077  1.0    1134.0  13060.20   3238.21   15856.10  1421.560   
4    TCGA-BA-6871  1.0     108.0   1960.77  11332.20   61040.20   402.572   
..            ...  ...       ...       ...       ...        ...       ...   
147  TCGA-QK-AA3K  0.0     253.0   2242.77   3855.33  210930.00   858.857   
148  TCGA-T2-A6WZ  1.0     229.0   5483.76   1669.65    6987.68  4541.990   
149  TCGA-UF-A7JS  1.0     519.0  16310.90   2346.79    3641.26  7357.250   
150  TCGA-UP-A6WW  0.0     518.0  20406.10   5338.79   22111.50  1330.080   
151  TCGA-WA-A7H4  0.0     443.0   7153.04   1789.26   52455.90  2401.440   

        ITGAV  
0     4382.68  
1    17909.80  
2     7588.91  
3     6872.

## Z score

In [64]:
import numpy as np
import scipy.stats as stats

In [65]:
df_rna=os_surv_hnsc[["FID"]+[x for x in genes[0].tolist() if x in rna.columns.tolist()]]
df_rna=df_rna.set_index("FID")
print(df_rna)

                  SDC4     ICAM1        FN1      RHOD     ITGAV
FID                                                            
TCGA-4P-AA8J   7723.31   3734.40  122122.00  2278.850   4382.68
TCGA-BA-4074  17101.30   1989.34   95369.30  6075.160  17909.80
TCGA-BA-4075  29713.00   1375.32   42006.70  4672.650   7588.91
TCGA-BA-4077  13060.20   3238.21   15856.10  1421.560   6872.47
TCGA-BA-6871   1960.77  11332.20   61040.20   402.572   2968.81
...                ...       ...        ...       ...       ...
TCGA-QK-AA3K   2242.77   3855.33  210930.00   858.857   3313.34
TCGA-T2-A6WZ   5483.76   1669.65    6987.68  4541.990   1784.99
TCGA-UF-A7JS  16310.90   2346.79    3641.26  7357.250   1756.35
TCGA-UP-A6WW  20406.10   5338.79   22111.50  1330.080   6651.45
TCGA-WA-A7H4   7153.04   1789.26   52455.90  2401.440  13270.80

[152 rows x 5 columns]


In [66]:
zdf=df_rna.apply(stats.zscore)
zdf['RHOD']=zdf['RHOD']*-1
#zdf['FSCN1']=zdf['FSCN1']*-1
print(zdf)
zdf['zscore_sum']=zdf.sum(axis=1)
zscore_sum=zdf['zscore_sum'].tolist()
os_surv_hnsc['zscore_sum']=zscore_sum
pfi_surv_hnsc['zscore_sum']=zscore_sum
print(os_surv_hnsc)
print(pfi_surv_hnsc)

                  SDC4     ICAM1       FN1      RHOD     ITGAV
FID                                                           
TCGA-4P-AA8J -0.271076  0.158347  0.395197  0.309330 -0.510373
TCGA-BA-4074  1.525685 -0.461650  0.186222 -1.294710  3.405268
TCGA-BA-4075  3.942004 -0.679803 -0.230614 -0.702113  0.417721
TCGA-BA-4077  0.751437 -0.017943 -0.434886  0.671557  0.210336
TCGA-BA-6871 -1.375141  2.857744 -0.081936  1.102106 -0.919640
...                ...       ...       ...       ...       ...
TCGA-QK-AA3K -1.321112  0.201312  1.088910  0.909314 -0.819910
TCGA-T2-A6WZ -0.700159 -0.575231 -0.504160 -0.646906 -1.262316
TCGA-UF-A7JS  1.374250 -0.334653 -0.530301 -1.836426 -1.270606
TCGA-UP-A6WW  2.158863  0.728365 -0.386023  0.710210  0.146359
TCGA-WA-A7H4 -0.380336 -0.532736 -0.148991  0.257532  2.062435

[152 rows x 5 columns]
              FID   OS  OS.time      SDC4     ICAM1        FN1      RHOD  \
0    TCGA-4P-AA8J  0.0    102.0   7723.31   3734.40  122122.00  2278.850   
1    

In [67]:
os_surv_hnsc["zscore_group"]=np.where(os_surv_hnsc["zscore_sum"]>os_surv_hnsc["zscore_sum"].quantile(0.8),"high","medium")
os_surv_hnsc["zscore_group"]=np.where(os_surv_hnsc["zscore_sum"]<os_surv_hnsc["zscore_sum"].quantile(0.2),"low",os_surv_hnsc["zscore_group"])
(os_surv_hnsc["zscore_group"]=="high").sum()
(os_surv_hnsc["zscore_group"]=="low").sum()

31

In [68]:
pfi_surv_hnsc["zscore_group"]=np.where(pfi_surv_hnsc["zscore_sum"]>pfi_surv_hnsc["zscore_sum"].quantile(0.8),"high","medium")
pfi_surv_hnsc["zscore_group"]=np.where(pfi_surv_hnsc["zscore_sum"]<pfi_surv_hnsc["zscore_sum"].quantile(0.2),"low",pfi_surv_hnsc["zscore_group"])
(pfi_surv_hnsc["zscore_group"]=="high").sum()
(pfi_surv_hnsc["zscore_group"]=="low").sum()

31

## Run Kaplan-Meier analysis

In [69]:
 def run_surv_plot(surv_df,surv_type,col):
        fig=plt.figure(figsize=(12,5))
        ax1 = plt.subplot(1,2,1)
            
        groups = surv_df[col]
        ix_high = (groups == "high")
        #ix_med = (groups == "medium")
        ix_low = (groups == "low")

        kmf = KaplanMeierFitter()
        kmf.fit(surv_df[surv_type+'.time'][ix_high], surv_df[surv_type][ix_high],label="Learn (n=31)")
        kmf.plot(color='green', ci_show=False, ax=ax1)
            
        kmf.fit(surv_df[surv_type+'.time'][ix_low], surv_df[surv_type][ix_low],label="Don't Learn/Forget (n=31)")
        kmf.plot(ax=ax1, color='orange', ci_show=False)
        
        #kmf.fit(surv_df[surv_type+'.time'][ix_med], surv_df[surv_type][ix_med],label="medium")
        #kmf.plot(ax=ax1, color='red', ci_show=False)
    

        results = logrank_test(surv_df[surv_type+'.time'][ix_high], surv_df[surv_type+'.time'][ix_low],event_observed_A=surv_df[surv_type][ix_high], event_observed_B=surv_df[surv_type][ix_low], alpha=.95) 
        #results = logrank_test(surv_df[surv_type+'.time'][ix_high], surv_df[surv_type+'.time'][ix_med],event_observed_A=surv_df[surv_type][ix_high], event_observed_B=surv_df[surv_type][ix_med], alpha=.95)
        plt.title('{}\nLog-rank test: p<{:.3}'.format(col,results.p_value))

        plt.xlabel('Days')
        plt.ylabel("% survival")
        plt.legend(frameon=False)

        plt.tight_layout()

        plt.savefig(col+'.pdf')
        plt.close()
        

In [70]:
run_surv_plot(os_surv_hnsc,"OS","zscore_group")

In [71]:
run_surv_plot(pfi_surv_hnsc,"PFI","zscore_group")